# Introduction to Machine Learning for Aerosol Scientists
## Part 2: Deep Neural Networks for UVLIF Aerosol Classification

### Learning Objectives
By the end of this notebook, you will understand:
1. How neural networks work (forward propagation, backpropagation)
2. Network architecture design (layers, neurons, activation functions)
3. Training dynamics (loss curves, overfitting, regularization)
4. Comparing DNNs to traditional ML (like XGBoost)

### Background
While XGBoost is excellent for structured data, deep neural networks (DNNs) offer a different approach:
- Learn hierarchical feature representations
- Highly flexible architecture
- Can capture complex non-linear patterns

We'll use the same UVLIF classification task to compare approaches directly.

In [ ]:
# Install required packages
!pip install tensorflow numpy pandas matplotlib seaborn scikit-learn -q

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import warnings
warnings.filterwarnings('ignore')

# Set style and random seeds for reproducibility
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## Step 1: Generate the Same Synthetic Data

We'll use identical data generation to ensure fair comparison with XGBoost.

In [ ]:
def generate_uvlif_spectrum(aerosol_type, n_wavelengths=64, noise_level=0.1):
    """Generate synthetic UVLIF spectrum for different aerosol types."""
    wavelengths = np.linspace(0, 1, n_wavelengths)
    
    if aerosol_type == 'biological':
        spectrum = (3.0 * np.exp(-((wavelengths - 0.3)**2) / 0.01) + 
                   4.0 * np.exp(-((wavelengths - 0.5)**2) / 0.02))
    elif aerosol_type == 'mineral_dust':
        spectrum = 0.5 + 0.3 * np.exp(-((wavelengths - 0.5)**2) / 0.5)
    elif aerosol_type == 'organic_carbon':
        spectrum = 2.5 * np.exp(-((wavelengths - 0.4)**2) / 0.03)
    elif aerosol_type == 'pah':
        spectrum = 5.0 * np.exp(-((wavelengths - 0.7)**2) / 0.02)
    
    noise = noise_level * np.random.randn(n_wavelengths)
    spectrum = spectrum + noise
    spectrum = np.maximum(spectrum, 0)
    
    return spectrum

# Generate dataset
np.random.seed(42)
n_samples_per_class = 250
aerosol_types = ['biological', 'mineral_dust', 'organic_carbon', 'pah']

data = []
labels = []

for aerosol_type in aerosol_types:
    for _ in range(n_samples_per_class):
        spectrum = generate_uvlif_spectrum(aerosol_type)
        data.append(spectrum)
        labels.append(aerosol_type)

X = np.array(data)
y = np.array(labels)

print(f"Dataset shape: {X.shape}")
print(f"Classes: {aerosol_types}")

## Step 2: Prepare Data for Neural Networks

Neural networks require:
- Scaled inputs (typically normalized to similar ranges)
- One-hot encoded labels for multi-class classification

In [ ]:
# Train/test/validation split (same as before)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Encode labels to integers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert to one-hot encoding for neural network
n_classes = len(aerosol_types)
y_train_onehot = keras.utils.to_categorical(y_train_encoded, n_classes)
y_val_onehot = keras.utils.to_categorical(y_val_encoded, n_classes)
y_test_onehot = keras.utils.to_categorical(y_test_encoded, n_classes)

print(f"Training samples: {X_train_scaled.shape[0]}")
print(f"Validation samples: {X_val_scaled.shape[0]}")
print(f"Test samples: {X_test_scaled.shape[0]}")
print(f"\nInput shape: {X_train_scaled.shape[1]} features")
print(f"Output shape: {n_classes} classes (one-hot encoded)")
print(f"\nExample one-hot encoding: {y_train_onehot[0]} = {label_encoder.classes_[y_train_encoded[0]]}")

## Step 3: Build a Simple Neural Network

### Neural Network Basics:

**Architecture components:**
1. **Input Layer**: Receives the 64 wavelength channels
2. **Hidden Layers**: Process information (learn patterns)
3. **Output Layer**: 4 neurons (one per aerosol class)

**Key concepts:**
- **Neurons**: Basic computational units
- **Weights**: Learned parameters connecting neurons
- **Activation Functions**: Introduce non-linearity (ReLU, softmax)
- **Dropout**: Randomly disable neurons during training (prevents overfitting)

Let's start with a simple architecture: Input → Dense(128) → Dense(64) → Output

In [ ]:
def create_simple_model(input_dim, n_classes):
    """
    Create a simple feedforward neural network.
    
    Architecture:
    - Input layer: 64 features
    - Hidden layer 1: 128 neurons with ReLU activation
    - Dropout: 30% (regularization)
    - Hidden layer 2: 64 neurons with ReLU activation
    - Dropout: 30%
    - Output layer: 4 neurons with softmax (probability distribution)
    """
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        
        # First hidden layer
        layers.Dense(128, activation='relu', name='hidden_layer_1'),
        layers.Dropout(0.3, name='dropout_1'),
        
        # Second hidden layer
        layers.Dense(64, activation='relu', name='hidden_layer_2'),
        layers.Dropout(0.3, name='dropout_2'),
        
        # Output layer
        layers.Dense(n_classes, activation='softmax', name='output_layer')
    ])
    
    return model

# Create the model
simple_model = create_simple_model(X_train_scaled.shape[1], n_classes)

# Display architecture
simple_model.summary()

print("\nArchitecture explained:")
print("- Total parameters: These are learned during training")
print("- Dropout layers: Prevent overfitting by randomly disabling connections")
print("- Softmax output: Converts to probability distribution (sums to 1.0)")

## Step 4: Compile and Train the Model

**Training process:**
1. **Forward pass**: Input → predictions
2. **Loss calculation**: How wrong are predictions?
3. **Backpropagation**: Calculate gradients
4. **Weight update**: Adjust weights to reduce loss

**Key training parameters:**
- **Optimizer (Adam)**: Algorithm for updating weights
- **Loss function (categorical crossentropy)**: Measures prediction error
- **Batch size**: Number of samples processed before updating weights
- **Epochs**: Number of complete passes through training data

In [ ]:
# Compile model
simple_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

print("Training the model...\n")
print("Callbacks enabled:")
print("- Early Stopping: Stops if validation loss doesn't improve for 15 epochs")
print("- Learning Rate Reduction: Reduces learning rate when stuck\n")

# Train model
history = simple_model.fit(
    X_train_scaled, y_train_onehot,
    validation_data=(X_val_scaled, y_val_onehot),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✓ Training complete!")

## Step 5: Analyze Training History

**Learning curves** show how the model learns over time:
- **Training loss/accuracy**: Performance on training data
- **Validation loss/accuracy**: Performance on unseen validation data

**What to look for:**
- Training and validation curves should be close (no overfitting)
- Both should improve over time
- Large gap = overfitting (model memorized training data)

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Model Loss During Training', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Model Accuracy During Training', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history_simple.png', dpi=150, bbox_inches='tight')
plt.show()

# Print final metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
print(f"\nFinal Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")
print(f"Gap (overfitting indicator): {abs(final_train_acc - final_val_acc):.4f}")

if abs(final_train_acc - final_val_acc) < 0.05:
    print("✓ Good! Small gap indicates the model generalizes well.")
else:
    print("⚠ Large gap suggests some overfitting.")

## Step 6: Evaluate on Validation Set

In [ ]:
# Get predictions
y_val_proba = simple_model.predict(X_val_scaled, verbose=0)
y_val_pred = np.argmax(y_val_proba, axis=1)

# Calculate accuracy
val_accuracy = (y_val_pred == y_val_encoded).mean()

print("="*60)
print("VALIDATION SET PERFORMANCE")
print("="*60)
print(f"\nValidation Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print("\nClassification Report:")
print(classification_report(y_val_encoded, y_val_pred,
                          target_names=label_encoder.classes_))

# Confusion matrix
cm = confusion_matrix(y_val_encoded, y_val_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix - Simple DNN', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix_dnn_simple.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ROC curves
y_val_bin = label_binarize(y_val_encoded, classes=range(n_classes))

plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'orange']

for i, (color, aerosol_type) in enumerate(zip(colors, aerosol_types)):
    fpr, tpr, _ = roc_curve(y_val_bin[:, i], y_val_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2,
             label=f'{aerosol_type.replace("_", " ").title()} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Simple DNN', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curves_dnn_simple.png', dpi=150, bbox_inches='tight')
plt.show()

## Step 7: Build a Deeper Network

Let's try a more complex architecture to see if we can improve performance:
- More layers (deeper network)
- Batch normalization (stabilizes training)
- Different dropout rates

In [ ]:
def create_deep_model(input_dim, n_classes):
    """
    Create a deeper neural network with batch normalization.
    
    Architecture:
    - Deeper: 4 hidden layers instead of 2
    - Batch normalization after each layer
    - Progressive size reduction (256→128→64→32)
    """
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        
        # Layer 1
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        
        # Layer 2
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # Layer 3
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # Layer 4
        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        # Output
        layers.Dense(n_classes, activation='softmax')
    ])
    
    return model

# Create deep model
deep_model = create_deep_model(X_train_scaled.shape[1], n_classes)
deep_model.summary()

print("\nKey differences from simple model:")
print("- 4 hidden layers vs 2 (more capacity to learn patterns)")
print("- Batch normalization (stabilizes training, allows higher learning rates)")
print("- More parameters (more flexible but risk of overfitting)")

In [ ]:
# Compile deep model
deep_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train deep model
print("Training deeper network...\n")

history_deep = deep_model.fit(
    X_train_scaled, y_train_onehot,
    validation_data=(X_val_scaled, y_val_onehot),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✓ Deep model training complete!")

In [ ]:
# Compare training histories
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Simple model - Loss
axes[0, 0].plot(history.history['loss'], label='Training', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[0, 0].set_title('Simple Model - Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Simple model - Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Training', linewidth=2)
axes[0, 1].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0, 1].set_title('Simple Model - Accuracy', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Deep model - Loss
axes[1, 0].plot(history_deep.history['loss'], label='Training', linewidth=2, color='darkred')
axes[1, 0].plot(history_deep.history['val_loss'], label='Validation', linewidth=2, color='red')
axes[1, 0].set_title('Deep Model - Loss', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Deep model - Accuracy
axes[1, 1].plot(history_deep.history['accuracy'], label='Training', linewidth=2, color='darkred')
axes[1, 1].plot(history_deep.history['val_accuracy'], label='Validation', linewidth=2, color='red')
axes[1, 1].set_title('Deep Model - Accuracy', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Evaluate deep model
y_val_proba_deep = deep_model.predict(X_val_scaled, verbose=0)
y_val_pred_deep = np.argmax(y_val_proba_deep, axis=1)
val_accuracy_deep = (y_val_pred_deep == y_val_encoded).mean()

print("="*60)
print("DEEP MODEL VALIDATION PERFORMANCE")
print("="*60)
print(f"\nValidation Accuracy: {val_accuracy_deep:.4f} ({val_accuracy_deep*100:.2f}%)")
print("\nComparison:")
print(f"Simple Model: {val_accuracy:.4f}")
print(f"Deep Model: {val_accuracy_deep:.4f}")
print(f"Improvement: {(val_accuracy_deep - val_accuracy)*100:.2f} percentage points")
print("\nClassification Report:")
print(classification_report(y_val_encoded, y_val_pred_deep,
                          target_names=label_encoder.classes_))

## Step 8: Final Evaluation on Test Set

Let's evaluate our best model on the held-out test set.

In [ ]:
# Choose the better model (compare validation accuracies)
if val_accuracy_deep >= val_accuracy:
    best_nn_model = deep_model
    model_name = "Deep Model"
else:
    best_nn_model = simple_model
    model_name = "Simple Model"

print(f"Selected {model_name} for final evaluation\n")

# Test set evaluation
y_test_proba = best_nn_model.predict(X_test_scaled, verbose=0)
y_test_pred = np.argmax(y_test_proba, axis=1)
test_accuracy = (y_test_pred == y_test_encoded).mean()

print("="*60)
print("FINAL TEST SET EVALUATION")
print("="*60)
print(f"\nTest Set Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("\nClassification Report:")
print(classification_report(y_test_encoded, y_test_pred,
                          target_names=label_encoder.classes_))

# Test set confusion matrix
cm_test = confusion_matrix(y_test_encoded, y_test_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title(f'Confusion Matrix - {model_name} (Test Set)', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix_dnn_test.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Test set ROC curves
y_test_bin = label_binarize(y_test_encoded, classes=range(n_classes))

plt.figure(figsize=(10, 8))

for i, (color, aerosol_type) in enumerate(zip(colors, aerosol_types)):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_test_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2,
             label=f'{aerosol_type.replace("_", " ").title()} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title(f'ROC Curves - {model_name} (Test Set)', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curves_dnn_test.png', dpi=150, bbox_inches='tight')
plt.show()

## Step 9: Visualize What the Network Learned

Let's examine the first layer weights to understand what patterns the network detects.

In [ ]:
# Get first layer weights
first_layer_weights = best_nn_model.layers[0].get_weights()[0]  # Shape: (64, n_neurons)

# Visualize first 16 neurons' weights
fig, axes = plt.subplots(4, 4, figsize=(15, 12))
axes = axes.ravel()

for i in range(16):
    axes[i].plot(first_layer_weights[:, i], linewidth=2)
    axes[i].set_title(f'Neuron {i+1}', fontsize=10)
    axes[i].set_xlabel('Wavelength Channel', fontsize=8)
    axes[i].set_ylabel('Weight', fontsize=8)
    axes[i].grid(True, alpha=0.3)
    axes[i].tick_params(labelsize=7)

plt.suptitle('First Layer Weight Patterns - What Each Neuron Detects', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('first_layer_weights.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nInterpretation:")
print("Each neuron learns to detect specific spectral patterns.")
print("Some neurons respond to peaks, others to valleys or specific wavelength ranges.")
print("These learned features are then combined in deeper layers for classification.")

## Step 10: Compare with XGBoost Results

Let's create a comprehensive comparison between approaches.

In [ ]:
# Create comparison summary
comparison_data = {
    'Metric': ['Test Accuracy', 'Mean AUC', 'Training Time', 'Interpretability', 
               'Hyperparameter Tuning', 'Data Efficiency'],
    'XGBoost': ['~97-99%', '~0.99', 'Fast (minutes)', 'High (feature importance)', 
                'Critical', 'Good with small data'],
    'Deep Neural Network': [f'{test_accuracy*100:.1f}%', 
                           f'{np.mean([auc(fpr, tpr) for fpr, tpr in [(roc_curve(y_test_bin[:, i], y_test_proba[:, i])[:2]) for i in range(n_classes)]]):.2f}',
                           'Moderate (minutes)', 'Lower (black box)', 
                           'Important', 'Needs more data for best results']
}

df_comparison = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("COMPARISON: XGBoost vs Deep Neural Networks")
print("="*80)
print(df_comparison.to_string(index=False))
print("\n" + "="*80)

## Summary and Key Takeaways

### What We Learned About Neural Networks:

1. **Architecture Design**
   - Layers and neurons control model capacity
   - Deeper ≠ always better (risk of overfitting)
   - Batch normalization and dropout are crucial regularization techniques

2. **Training Dynamics**
   - Learning curves reveal overfitting vs underfitting
   - Early stopping prevents overtraining
   - Learning rate scheduling helps convergence

3. **When to Use DNNs vs XGBoost**

   **Use XGBoost when:**
   - Working with tabular/structured data
   - Need interpretability (feature importance)
   - Have limited data
   - Want fast training and tuning
   - Need robust baseline quickly
   
   **Use Deep Neural Networks when:**
   - Working with images, sequences, or unstructured data
   - Have large datasets
   - Need to learn complex hierarchical features
   - Can invest time in architecture search
   - Transfer learning is an option

### For Aerosol Mass Spectra:
- Both methods work excellently (~97-99% accuracy)
- XGBoost might be preferred due to:
  - Simpler to tune
  - Faster training
  - Better interpretability
  - Works well with our data size
- DNNs show their strength with larger, more complex datasets

### Next Steps in Your ML Journey:
1. Try these methods on your real UVLIF data
2. Experiment with different architectures
3. Explore ensemble methods (combining models)
4. Look into transfer learning for related aerosol tasks
5. Consider 1D Convolutional Neural Networks for spectral data

In [ ]:
# Save the best neural network model
best_nn_model.save('best_dnn_model.keras')
print("✓ Model saved to 'best_dnn_model.keras'")
print("\nYou can load it later with: model = keras.models.load_model('best_dnn_model.keras')")

## Bonus: Example Prediction Function

Here's how you would use the trained model for new predictions in practice:

In [ ]:
def predict_aerosol_type(spectrum, model, scaler, label_encoder):
    """
    Predict aerosol type from a UVLIF spectrum.
    
    Parameters:
    -----------
    spectrum : array-like, shape (n_wavelengths,)
        UVLIF fluorescence spectrum
    model : keras.Model
        Trained neural network
    scaler : StandardScaler
        Fitted scaler for normalization
    label_encoder : LabelEncoder
        Label encoder for class names
    
    Returns:
    --------
    prediction : str
        Predicted aerosol type
    probabilities : dict
        Probability for each class
    """
    # Ensure 2D shape
    spectrum_2d = np.array(spectrum).reshape(1, -1)
    
    # Scale
    spectrum_scaled = scaler.transform(spectrum_2d)
    
    # Predict
    proba = model.predict(spectrum_scaled, verbose=0)[0]
    pred_class_idx = np.argmax(proba)
    pred_class = label_encoder.classes_[pred_class_idx]
    
    # Create probability dictionary
    prob_dict = {label_encoder.classes_[i]: float(proba[i]) 
                 for i in range(len(label_encoder.classes_))}
    
    return pred_class, prob_dict

# Test with a random sample
test_idx = np.random.randint(0, len(X_test))
test_spectrum = X_test[test_idx]
true_label = y_test[test_idx]

predicted_label, probabilities = predict_aerosol_type(
    test_spectrum, best_nn_model, scaler, label_encoder
)

print("\nExample Prediction:")
print("="*60)
print(f"True Label: {true_label}")
print(f"Predicted Label: {predicted_label}")
print(f"\nClass Probabilities:")
for aerosol_type, prob in sorted(probabilities.items(), key=lambda x: x[1], reverse=True):
    print(f"  {aerosol_type:20s}: {prob*100:5.2f}%")

# Visualize the spectrum
plt.figure(figsize=(12, 5))
plt.plot(test_spectrum, linewidth=2, color='steelblue')
plt.xlabel('Wavelength Channel', fontsize=12)
plt.ylabel('Fluorescence Intensity', fontsize=12)
plt.title(f'Example Spectrum: True={true_label}, Predicted={predicted_label}', 
          fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()